In [1]:
# Importaciones
import pandas as pd
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
import numpy as np
from sklearn import preprocessing
import seaborn as sns
from datetime import datetime
from IPython.core.display import display, HTML
from scipy import stats
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
data=pd.read_csv(r'C:\Users\alexb\Documents\Facultad\UBA\Modelo_PGSM_3T\Datos\20210811 - Segunda corrida de datos\To_FS_20210913_2137_3T.csv', delimiter=',', sep='\n', decimal='.', index_col='Time')
data.head()

,Unnamed: 0,LT-17,Conc_P3-4_MVA,T_Lam_9_MVA,Conc_P1,Humedad_Semilla_MVA,Vel_Extractor,T_Mat_entrada_MVA,TT-35-GI,MG_NIR_Salida_Ext_MVA,T_Solvente_MVA,CP1_/_HumedadMat,CP1_/_TLam,CP34_/_CP1
Time,,,,,,,,,,,,,,
2021-07-25 00:05:01,0,-0.459938,0.3857,2.091497,0.192989,0.926836,0.036691,-0.360824,1.133000,-1.167139,0.407761,0.208224,0.092273,1.998557
2021-07-25 00:05:02,1,-0.459938,0.3857,2.091240,0.192989,0.926836,0.036691,-0.359523,1.088869,-1.166637,0.412988,0.208224,0.092285,1.998557
2021-07-25 00:05:03,2,-0.460231,0.3857,2.090983,0.192989,0.927124,0.036691,-0.358376,1.133000,-1.166051,0.417819,0.208159,0.092296,1.998557
2021-07-25 00:05:04,3,-0.460231,0.3857,2.090725,0.192989,0.927989,0.036691,-0.357764,1.133000,-1.165389,0.422649,0.207965,0.092307,1.998557
2021-07-25 00:05:05,4,-0.459938,0.3857,2.090468,0.192989,0.928853,0.036691,-0.357382,1.133000,-1.164659,0.427479,0.207772,0.092319,1.998557


In [12]:
data.drop(columns = 'Unnamed: 0', inplace = True)

In [19]:
X = data.drop(columns = 'MG_NIR_Salida_Ext_MVA')
y = data.MG_NIR_Salida_Ext_MVA

# SELECT FROM MODEL (SFM)

**SFM** o **SELECT FROM MODEL** es un método de selección de features que utiliza las estimaciones de un modelo (al igual que RFE). La diferencia es que en este caso la selección de las features se hace estableciendo un umbral para el atributo que se está evaluando (coeficientes o feature importances).

In [21]:
from sklearn.ensemble import GradientBoostingRegressor

In [16]:
from sklearn.feature_selection import SelectFromModel

In [22]:
sfm_selector = SelectFromModel(estimator = GradientBoostingRegressor())

In [23]:
sfm_selector.fit(X, y)

SelectFromModel(estimator=GradientBoostingRegressor())

In [24]:
X.columns[sfm_selector.get_support()]

Index(['Conc_P3-4_MVA', 'T_Lam_9_MVA', 'Conc_P1', 'T_Mat_entrada_MVA'], dtype='object')

# RECURSIVE FEATURE ELIMINATION (RFE)

**RFE** o **RECURSIVE FEATURE ELIMINATION** es un método de selección de características utilizado en ML en donde se elimina la característica menos importantes luego de entrenar de forma recursiva.

Se comienza entrenando en el conjunto completo de features y se obtiene la importancia de cada feature mediante los coeficientes del modelo. En base a eso, la feature con menos importancia es eliminada.
El proceso se repite hasta alcanzar el numero deseado de características.

In [25]:
from sklearn.feature_selection import RFE

In [26]:
rfe_selector = RFE(estimator = GradientBoostingRegressor(), n_features_to_select = 4, step = 1) # step es la cantidad de features que se descartan por iteración

In [27]:
rfe_selector.fit(X, y)

RFE(estimator=GradientBoostingRegressor(), n_features_to_select=4)

In [28]:
X.columns[rfe_selector.get_support()]

Index(['Conc_P3-4_MVA', 'T_Lam_9_MVA', 'Conc_P1', 'T_Mat_entrada_MVA'], dtype='object')

# SEQUENTIAL FEATURE SELECTION (SFS)

**SFE** o **SEQUENTIAL FEATURE SELECTION** es un algorítmo de selección de features que utiliza el puntaje obtenido con el CV set como estimador para seleccionar las mejores features.
Existen dos variantes de este algorítmo: SFS-Forward y SFS-Backward.

En el caso de SFS-Forward, se comienza con una única feature (la que mejor resultado brinda por si sola en el CV set). Luego se van seleccionando de a una, las variables que mejor se desempeñan, hasta alcanzar el numero de características deseadas.

Notar que en este caso **no se usan los atributos del modelo: coef_ ni feature_importances_**. Por otro lado, este algorítmo es más lento, ya que resulta de entrenar el modelo múltiples veces.

In [30]:
from sklearn.feature_selection import SequentialFeatureSelector

In [37]:
sfs_selector = SequentialFeatureSelector(estimator = GradientBoostingRegressor(), n_features_to_select = 4, cv = 4, direction = 'forward')

In [38]:
sfs_selector.fit(X, y)

SequentialFeatureSelector(cv=4, estimator=GradientBoostingRegressor(),
                          n_features_to_select=4)

In [39]:
X.columns[sfs_selector.get_support()]

Index(['Vel_Extractor', 'T_Mat_entrada_MVA', 'TT-35-GI', 'T_Solvente_MVA'], dtype='object')

In [42]:
sfs_selector.get_params()

{'cv': 4,
 'direction': 'forward',
 'estimator__alpha': 0.9,
 'estimator__ccp_alpha': 0.0,
 'estimator__criterion': 'friedman_mse',
 'estimator__init': None,
 'estimator__learning_rate': 0.1,
 'estimator__loss': 'ls',
 'estimator__max_depth': 3,
 'estimator__max_features': None,
 'estimator__max_leaf_nodes': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_iter_no_change': None,
 'estimator__random_state': None,
 'estimator__subsample': 1.0,
 'estimator__tol': 0.0001,
 'estimator__validation_fraction': 0.1,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': GradientBoostingRegressor(),
 'n_features_to_select': 4,
 'n_jobs': None,
 'scoring': None}